# Project 3: Smart Beta Portfolio and Portfolio Optimization

## Overview


Smart beta has a broad meaning, but we can say in practice that when we use the universe of stocks from an index, and then apply some weighting scheme other than market cap weighting, it can be considered a type of smart beta fund.  A Smart Beta portfolio generally gives investors exposure or "beta" to one or more types of market characteristics (or factors) that are believed to predict prices while giving investors a diversified broad exposure to a particular market. Smart Beta portfolios generally target momentum, earnings quality, low volatility, and dividends or some combination. Smart Beta Portfolios are generally rebalanced infrequently and follow relatively simple rules or algorithms that are passively managed.  Model changes to these types of funds are also rare requiring prospectus filings with US Security and Exchange Commission in the case of US focused mutual funds or ETFs.. Smart Beta portfolios are generally long-only, they do not short stocks.

In contrast, a purely alpha-focused quantitative fund may use multiple models or algorithms to create a portfolio. The portfolio manager retains discretion in upgrading or changing the types of models and how often to rebalance the portfolio in attempt to maximize performance in comparison to a stock benchmark.  Managers may have discretion to short stocks in portfolios.

Imagine you're a portfolio manager, and wish to try out some different portfolio weighting methods.

One way to design portfolio is to look at certain accounting measures (fundamentals) that, based on past trends, indicate stocks that produce better results.  


For instance, you may start with a hypothesis that dividend-issuing stocks tend to perform better than stocks that do not. This may not always be true of all companies; for instance, Apple does not issue dividends, but has had good historical performance.  The hypothesis about dividend-paying stocks may go something like this: 

Companies that regularly issue dividends may also be more prudent in allocating their available cash, and may indicate that they are more conscious of prioritizing shareholder interests.  For example, a CEO may decide to reinvest cash into pet projects that produce low returns.  Or, the CEO may do some analysis, identify that reinvesting within the company produces lower returns compared to a diversified portfolio, and so decide that shareholders would be better served if they were given the cash (in the form of dividends).  So according to this hypothesis, dividends may be both a proxy for how the company is doing (in terms of earnings and cash flow), but also a signal that the company acts in the best interest of its shareholders.  Of course, it's important to test whether this works in practice.


You may also have another hypothesis, with which you wish to design a portfolio that can then be made into an ETF.  You may find that investors may wish to invest in passive beta funds, but wish to have less risk exposure (less volatility) in their investments.  The goal of having a low volatility fund that still produces returns similar to an index may be appealing to investors who have a shorter investment time horizon, and so are more risk averse.

So the objective of your proposed portfolio is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  Also, if this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

Smart Beta ETFs can be designed with both of these two general methods (among others): alternative weighting and minimum volatility ETF.


## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment. After implementing the function, run the cell to test it against the unit tests we've provided. For each problem, we provide one or more unit tests from our `project_tests` package. These unit tests won't tell you if your answer is correct, but will warn you of any major errors. Your code will be checked for the correct solution when you submit it to Udacity.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

The other packages that we're importing are `helper`, `project_helper`, and `project_tests`. These are custom packages built to help you solve the problems.  The `helper` and `project_helper` module contains utility functions and graph functions. The `project_tests` contains the unit tests for all the problems.
### Install Packages

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

### Load Packages

In [2]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [3]:
df = pd.read_csv('../../data/project_3/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [4]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
In Part 1 of this project, you'll build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF.  You'll compare this portfolio to a market cap weighted index to see how well it performs. 

Note that in practice, you'll probably get the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's), but for this exercise we will simulate a market cap weighted index.

## Index Weights
The index we'll be using is based on large dollar volume stocks. Implement `generate_dollar_volume_weights` to generate the weights for this index. For each date, generate the weights based on dollar volume traded for that date. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [5]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    #TODO: Implement function
    dollar_vol = close*volume
    print(dollar_vol)
    print('*'*80)
    
    dollar_vol ['date_total_volume'] = dollar_vol.sum(axis = 1)
    print(dollar_vol)
    print('*'*80)
    
    
    dollar_volume_weights = dollar_vol.iloc[:,:-1].div(dollar_vol.iloc[:,-1], axis = 0) #?axis = 0 across index
    print(dollar_volume_weights)

    return dollar_volume_weights

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

                           MRE                YIMJ                 WTS
2006-02-01  348628075.71300000  608648315.28000009  300410784.98600000
2006-02-02 7575630049.36999989 6240087760.44999981 4380507847.72000027
2006-02-03  941144343.83999991  754203279.77999997  553581860.38199997
2006-02-04  364742751.09999996 5254224178.90000057  545086266.30700004
********************************************************************************
                           MRE                YIMJ                 WTS  \
2006-02-01  348628075.71300000  608648315.28000009  300410784.98600000   
2006-02-02 7575630049.36999989 6240087760.44999981 4380507847.72000027   
2006-02-03  941144343.83999991  754203279.77999997  553581860.38199997   
2006-02-04  364742751.09999996 5254224178.90000057  545086266.30700004   

              date_total_volume  
2006-02-01  1257687175.97900009  
2006-02-02 18196225657.54000092  
2006-02-03  2248929484.00199986  
2006-02-04  6164053196.30700111  
***********************

### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [6]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

ticker                    AAL                 AAPL               ABBV  \
date                                                                    
2013-07-01 202391976.65295082  5192133342.44938660 160240491.76812646   
2013-07-02 170044135.53290918  6379858749.09848881 154675436.02818424   
2013-07-03 113535574.39577270  3289403473.24362564  85687954.74287605   
2013-07-05 104208202.16722879  3711212490.37529945  98378473.62004057   
2013-07-08 116808737.36893333  4014865794.27665520 145851994.43971246   
2013-07-09 182685199.45438606  4831570151.80062866 138374228.00378600   
2013-07-10 190130912.43986583  3841394533.31340551 113516885.36293092   
2013-07-11 141413052.51851743  4523559972.82930374 150603022.78256321   
2013-07-12 178919709.25073317  3868672078.74982738 217263513.19007871   
2013-07-15 149777857.10430652  3355016082.93688917 123424483.27640234   
2013-07-16 177889120.79958883  3022401649.64293003 137833816.92883056   
2013-07-17 299658647.97497630  2778217622.68652296 

## Portfolio Weights
Now that we have the index weights, let's choose the portfolio weights based on dividend. You would normally calculate the weights based on trailing dividend yield, but we'll simplify this by just calculating the total dividend yield over time.

Implement `calculate_dividend_weights` to return the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.
For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [7]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    print('dividends')
    print(dividends)
    print('*'*80)
       
#     dividends['date_total'] = dividends.sum(axis = 1) #axis = 1 across the columns
#     print(dividends)
#     print('*'*80)
    
    total_div_to_date = dividends.cumsum() #running total of dividend per ticker
    total_div_to_date['universe_total'] = total_div_to_date.sum(axis = 1)
    print(total_div_to_date)
    print('*'*80)
    
    dividend_weights = total_div_to_date.iloc[:,:-1].div(total_div_to_date.iloc[:,-1], axis = 0) #? axis = 0 = along index
    print(type(total_div_to_date.iloc[:,:-1]))
    print(dividend_weights)
    
    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

dividends
                  FDD       QHTM        PYE
2012-07-26 0.00000000 0.00000000 0.00000000
2012-07-27 0.00000000 0.00000000 0.10000000
2012-07-28 0.00000000 1.00000000 0.30000000
2012-07-29 0.00000000 0.20000000 0.00000000
********************************************************************************
                  FDD       QHTM        PYE  universe_total
2012-07-26 0.00000000 0.00000000 0.00000000      0.00000000
2012-07-27 0.00000000 0.00000000 0.10000000      0.10000000
2012-07-28 0.00000000 1.00000000 0.40000000      1.40000000
2012-07-29 0.00000000 1.20000000 0.40000000      1.60000000
********************************************************************************
<class 'pandas.core.frame.DataFrame'>
                  FDD       QHTM        PYE
2012-07-26        nan        nan        nan
2012-07-27 0.00000000 0.00000000 1.00000000
2012-07-28 0.00000000 0.71428571 0.28571429
2012-07-29 0.00000000 0.75000000 0.25000000
Tests Passed


### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [8]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

dividends
ticker            AAL       AAPL       ABBV        ABT        AGN        AIG  \
date                                                                           
2013-07-01 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-02 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-03 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-05 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-08 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-09 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-10 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-11 0.00000000 0.00000000 0.40000000 0.14000000 0.00000000 0.00000000   
2013-07-12 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-15 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000   
2013-07-16 0.00000000 0.000000

## Returns
Implement `generate_returns` to generate returns data for all the stocks and dates from price data. You might notice we're implementing returns and not log returns. Since we're not dealing with volatility, we don't have to use log returns.

In [9]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function
    print(prices.head())
    print('*'*80)
    print(prices - prices.shift(1)) #shift yesterday's prices 1 step forward then subtract from today's price
    print('*'*80)

    #return (prices - prices.shift(1)) / prices.shift(1)
    return (prices / prices.shift(1)) -1

project_tests.test_generate_returns(generate_returns)

                  HANX         AJC         MAR
1999-11-24 35.44110000 34.17990000 34.02230000
1999-11-25 92.11310000 91.05430000 90.95720000
1999-11-26 57.97080000 57.78140000 58.19820000
1999-11-27 34.17050000 92.45300000 58.51070000
********************************************************************************
                   HANX          AJC          MAR
1999-11-24          nan          nan          nan
1999-11-25  56.67200000  56.87440000  56.93490000
1999-11-26 -34.14230000 -33.27290000 -32.75900000
1999-11-27 -23.80030000  34.67160000   0.31250000
********************************************************************************
Tests Passed


### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [10]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

ticker             AAL        AAPL        ABBV         ABT          AGN  \
date                                                                      
2013-07-01 16.17609308 53.10917319 34.92447839 31.42538772 122.62751990   
2013-07-02 15.81983388 54.31224742 35.42807578 31.27288084 121.05361758   
2013-07-03 16.12794994 54.61204262 35.44486235 30.72565028 121.21003024   
2013-07-05 16.21460758 54.17338125 35.85613355 31.32670680 123.53666845   
2013-07-08 16.31089385 53.86579916 36.66188936 31.76628544 123.65397794   

ticker             AIG        AMAT        AMGN         AMZN         APC  \
date                                                                      
2013-07-01 41.55339742 13.63297558 86.80333359 282.10000000 82.90947737   
2013-07-02 41.36908428 13.63757665 85.53008744 283.73000000 82.89037160   
2013-07-03 40.75163526 13.80321523 85.30749196 284.03000000 83.04321774   
2013-07-05 41.64555399 14.00566239 86.93689088 285.88000000 84.42838587   
2013-07-08 41.94967067 1

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [11]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    #TODO: Implement function
    print(returns)
    print(weights)
    print(returns*weights)

    #return returns.mul(weights)
    return returns*weights

project_tests.test_generate_weighted_returns(generate_weighted_returns)

                  ALOA         AKS         DLN
2000-08-07         nan         nan         nan
2000-08-08  1.59904743  1.66397210  1.67345829
2000-08-09 -0.37065629 -0.36541822 -0.36015840
2000-08-10 -0.41055669  0.60004777  0.00536958
                 ALOA        AKS        DLN
2000-08-07 0.03777059 0.04733924 0.05197790
2000-08-08 0.82074874 0.48533938 0.75792752
2000-08-09 0.10196420 0.05866016 0.09578226
2000-08-10 0.03951647 0.40866122 0.09431233
                  ALOA         AKS         DLN
2000-08-07         nan         nan         nan
2000-08-08  1.31241616  0.80759119  1.26836009
2000-08-09 -0.03779367 -0.02143549 -0.03449679
2000-08-10 -0.01622375  0.24521625  0.00050642
Tests Passed


### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [12]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
2013-07-09  0.02479339  0.01758824 -0.00824176 -0.01892121 -0.00616650   
2013-07-10 -0.01094470 -0.00383568  0.01361958  0.00000000 -0.01304590   
2013-07-11  0.01164822  0.01558719  0.00614894  0.02216465  0.00749577   
2013-07-12  0.01093840 -0.00182079  0.02900868 -0.00141363  0.00944000   
2013-07-15  0.01138952  0.00218049 -0.00954495  0.00453001 -0.00792519   
2013-07-16  0.01069820  0.00644535 -0.01815329  0.00620068 -0.01869308   
2013-07-17  0.02785515  0.00026732  0.

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [13]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function
    #Cumulative arithmetic return
    #cum_log_return = log_returns.cumsum()[len(log_returns)-1]
    print(returns)
    print('*'*80)
    print(returns.cumsum(axis = 1))
#     print('*'*80)
#     print(np.cumprod(returns))
#     print('*'*80)
#     print(returns.cumprod(axis = 1)) #total returns for each date, across columns
#     print('*'*80)
#     print(returns.cumprod(axis = 1).iloc[:,-1])
#     print('*'*80)
#     print('*'*80)
    
    #return returns.cumprod(axis = 1).iloc[:,-1]
    #return np.cumprod((returns+1),axis = 1).iloc[:,-1] 
    #return returns.cumsum(axis = 1).iloc[:,-1]/returns.shape[0]
    #return ((returns+1).cumprod(axis = 1).iloc[:,-1] -1)
    daily_total_return = 1 + returns.cumsum(axis = 1).iloc[:,-1] 
    print(daily_total_return)
    cumulative_returns = daily_total_return.cumprod()
    print(cumulative_returns)
    
    return cumulative_returns

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

                   OGT        RNUA         TPM
2015-10-28         nan         nan         nan
2015-10-29  1.59904743  1.66397210  1.67345829
2015-10-30 -0.37065629 -0.36541822 -0.36015840
2015-10-31 -0.41055669  0.60004777  0.00536958
********************************************************************************
                   OGT        RNUA         TPM
2015-10-28         nan         nan         nan
2015-10-29  1.59904743  3.26301953  4.93647782
2015-10-30 -0.37065629 -0.73607451 -1.09623291
2015-10-31 -0.41055669  0.18949108  0.19486066
2015-10-28           nan
2015-10-29    5.93647782
2015-10-30   -0.09623291
2015-10-31    1.19486066
Name: TPM, dtype: float64
2015-10-28           nan
2015-10-29    5.93647782
2015-10-30   -0.57128454
2015-10-31   -0.68260542
Name: TPM, dtype: float64
Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [14]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00008007  0.00308984  0.00004768 -0.00001880 -0.00003423   
2013-07-03  0.00009093  0.00074662  0.00000167 -0.00018529  0.00000450   
2013-07-05  0.00001716 -0.00091335  0.00003497  0.00008758  0.00005552   
2013-07-08  0.00001590 -0.00052257  0.00007514  0.00007710  0.00000238   
2013-07-09  0.00009451  0.00177325 -0.00002380 -0.00012663 -0.00001420   
2013-07-10 -0.00004895 -0.00034661  0.00003637  0.00000000 -0.00004299   
2013-07-11  0.00003254  0.00139306  0.00001830  0.00012582  0.00002055   
2013-07-12  0.00003548 -0.00012769  0.00011425 -0.00000507  0.00002572   
2013-07-15  0.00003910  0.00016769 -0.00002700  0.00002153 -0.00002676   
2013-07-16  0.00004287  0.00043884 -0.00005637  0.00003814 -0.00005587   
2013-07-17  0.00018322  0.00001630  0.

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [15]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    print(benchmark_returns_by_date)
    print(np.sqrt(252)*np.std(etf_returns_by_date - benchmark_returns_by_date ))

    return np.sqrt(252)*np.std(etf_returns_by_date - benchmark_returns_by_date,ddof =1 )

project_tests.test_tracking_error(tracking_error)

2016-04-23          nan
2016-04-24   0.99880148
2016-04-25   0.99876653
2016-04-26   1.00024411
dtype: float64
13.493621066038921
Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [16]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

date
2013-07-01           nan
2013-07-02    0.00272899
2013-07-03    0.00204539
2013-07-05    0.00935141
2013-07-08    0.00442910
2013-07-09    0.00910603
2013-07-10   -0.00117523
2013-07-11    0.01521096
2013-07-12    0.00003842
2013-07-15    0.00416398
2013-07-16   -0.00220364
2013-07-17    0.00514664
2013-07-18    0.00329863
2013-07-19   -0.00896400
2013-07-22    0.00349631
2013-07-23   -0.00505016
2013-07-24    0.00482910
2013-07-25    0.06336125
2013-07-26    0.00437250
2013-07-29    0.00196704
2013-07-30    0.00887829
2013-07-31   -0.00507691
2013-08-01    0.01682491
2013-08-02    0.00565016
2013-08-05    0.00381798
2013-08-06   -0.00790239
2013-08-07   -0.00117250
2013-08-08    0.00473373
2013-08-09   -0.00485002
2013-08-12    0.00292309
                 ...    
2017-05-19    0.00715974
2017-05-22    0.00543233
2017-05-23    0.00249727
2017-05-24    0.00279225
2017-05-25    0.00433513
2017-05-26    0.00282244
2017-05-30   -0.00011467
2017-05-31   -0.00261092
2017-06-01    0.0066

# Part 2: Portfolio Optimization

Now, let's create a second portfolio.  We'll still reuse the market cap weighted index, but this will be independent of the dividend-weighted portfolio that we created in part 1.

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
Implement `get_covariance_returns` to calculate the covariance of the `returns`. We'll use this to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [17]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
    #print(returns)
    #print('*'*80)
    
    returns_copy = returns.copy()
    returns_copy = returns_copy.fillna(0)
    #print(returns_copy)
    #print('*'*80)
    
    
    #print(np.array(returns_copy).T) #each row is a now returns of a particular stock
    returns_covariance = np.cov(np.array(returns_copy).T)
    #print(returns_covariance)
    #print('*'*80)
    
    return returns_covariance

project_tests.test_get_covariance_returns(get_covariance_returns)

Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [18]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
Recall from the Pythagorean theorem that you can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  Can you guess what to pass into this function?



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [19]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    #TODO: Implement function
    #print(covariance_returns)
    #print('*'*80)
    #print(index_weights)
    #print('*'*80)
    
    x = cvx.Variable(len(covariance_returns))
    variance_portfolio = cvx.quad_form(x,covariance_returns)
    weights_deviation = cvx.norm(x - index_weights, p=2,axis = None)
    objective_function = cvx.Minimize(variance_portfolio + scale*weights_deviation)
    constraints = [x>= 0, sum(x)==1]
    problem_to_solve = cvx.Problem(objective_function, constraints)
    min_value = problem_to_solve.solve()
    #print(min_value)
    #print('*'*80)
    
    #print(x.value)
    #print('*'*80)
    
    optimal_weights = np.array(x.value)
    
    return optimal_weights 

project_tests.test_get_optimal_weights(get_optimal_weights)

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [20]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [21]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
2013-07-09  0.02479339  0.01758824 -0.00824176 -0.01892121 -0.00616650   
2013-07-10 -0.01094470 -0.00383568  0.01361958  0.00000000 -0.01304590   
2013-07-11  0.01164822  0.01558719  0.00614894  0.02216465  0.00749577   
2013-07-12  0.01093840 -0.00182079  0.02900868 -0.00141363  0.00944000   
2013-07-15  0.01138952  0.00218049 -0.00954495  0.00453001 -0.00792519   
2013-07-16  0.01069820  0.00644535 -0.01815329  0.00620068 -0.01869308   
2013-07-17  0.02785515  0.00026732  0.

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.00011130  0.00113194  0.00007320 -0.00001672 -0.00008104   
2013-07-03  0.00009843  0.00027582  0.00000241 -0.00006029  0.00000816   
2013-07-05  0.00002715 -0.00040137  0.00005890  0.00006740  0.00012121   
2013-07-08  0.00003001 -0.00028371  0.00011407  0.00004834  0.00000600   
2013-07-09  0.00012530  0.00087887 -0.00004184 -0.00006519 -0.00003894   
2013-07-10 -0.00005531 -0.00019167  0.00006914  0.00000000 -0.00008238   
2013-07-11  0.00005887  0.00077888  0.00003121  0.00007636  0.00004733   
2013-07-12  0.00005528 -0.00009098  0.00014726 -0.00000487  0.00005961   
2013-07-15  0.00005756  0.00010896 -0.00004845  0.00001561 -0.00005004   
2013-07-16  0.00005407  0.00032207 -0.00009215  0.00002136 -0.00011804   
2013-07-17  0.00014078  0.00001336  0.

date
2013-07-01           nan
2013-07-02    0.00272899
2013-07-03    0.00204539
2013-07-05    0.00935141
2013-07-08    0.00442910
2013-07-09    0.00910603
2013-07-10   -0.00117523
2013-07-11    0.01521096
2013-07-12    0.00003842
2013-07-15    0.00416398
2013-07-16   -0.00220364
2013-07-17    0.00514664
2013-07-18    0.00329863
2013-07-19   -0.00896400
2013-07-22    0.00349631
2013-07-23   -0.00505016
2013-07-24    0.00482910
2013-07-25    0.06336125
2013-07-26    0.00437250
2013-07-29    0.00196704
2013-07-30    0.00887829
2013-07-31   -0.00507691
2013-08-01    0.01682491
2013-08-02    0.00565016
2013-08-05    0.00381798
2013-08-06   -0.00790239
2013-08-07   -0.00117250
2013-08-08    0.00473373
2013-08-09   -0.00485002
2013-08-12    0.00292309
                 ...    
2017-05-19    0.00715974
2017-05-22    0.00543233
2017-05-23    0.00249727
2017-05-24    0.00279225
2017-05-25    0.00433513
2017-05-26    0.00282244
2017-05-30   -0.00011467
2017-05-31   -0.00261092
2017-06-01    0.0066

## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [22]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    all_rebalance_weights = []
    print(returns)
    print('*'*80)
    for end_index in range(chunk_size,len(returns),shift_size):
        start_index = end_index - chunk_size
        windowed_returns = returns.iloc[start_index:end_index,:]
        covariances = get_covariance_returns(windowed_returns)
        #index weights need to be a series here
        #had to adjust since using range for end index and iloc here .iloc[end_index-1,:]
        optimal_weights = get_optimal_weights(covariances, index_weights.iloc[end_index-1,:], scale = 2)
        all_rebalance_weights.append(optimal_weights)
    
    return all_rebalance_weights

project_tests.test_rebalance_portfolio(rebalance_portfolio)

                   GGO         GUF         QLN
2009-01-24         nan         nan         nan
2009-01-25 -0.02202381  0.02265285  0.01441961
2009-01-26  0.01947657  0.00551985  0.00047382
2009-01-27  0.00537313 -0.00803232  0.01160313
2009-01-28  0.00593824 -0.00567773  0.02247191
2009-01-29  0.02479339  0.01758824 -0.00824176
2009-01-30 -0.01094470 -0.00383568  0.01361958
2009-01-31  0.01164822  0.01558719  0.00614894
2009-02-01  0.01093840 -0.00182079  0.02900868
2009-02-02  0.01138952  0.00218049 -0.00954495
2009-02-03  0.01069820  0.00644535 -0.01815329
********************************************************************************
Tests Passed


Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [23]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

ticker             AAL        AAPL        ABBV         ABT         AGN  \
date                                                                     
2013-07-01         nan         nan         nan         nan         nan   
2013-07-02 -0.02202381  0.02265285  0.01441961 -0.00485298 -0.01283482   
2013-07-03  0.01947657  0.00551985  0.00047382 -0.01749857  0.00129209   
2013-07-05  0.00537313 -0.00803232  0.01160313  0.01956204  0.01919510   
2013-07-08  0.00593824 -0.00567773  0.02247191  0.01403207  0.00094959   
2013-07-09  0.02479339  0.01758824 -0.00824176 -0.01892121 -0.00616650   
2013-07-10 -0.01094470 -0.00383568  0.01361958  0.00000000 -0.01304590   
2013-07-11  0.01164822  0.01558719  0.00614894  0.02216465  0.00749577   
2013-07-12  0.01093840 -0.00182079  0.02900868 -0.00141363  0.00944000   
2013-07-15  0.01138952  0.00218049 -0.00954495  0.00453001 -0.00792519   
2013-07-16  0.01069820  0.00644535 -0.01815329  0.00620068 -0.01869308   
2013-07-17  0.02785515  0.00026732  0.

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover. We'll be using the formulas used in the classroom:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [24]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    print(all_rebalance_weights)
    sum_total_turnover = float(0)
    for t in range(1,len(all_rebalance_weights)):
        sum_total_turnover += np.sum(np.abs(all_rebalance_weights[t] - all_rebalance_weights[t-1]))
    
    num_events_per_year = n_trading_days_in_year / shift_size
    
    return (sum_total_turnover / rebalance_count)*num_events_per_year 

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

[array([1.22050335e-04, 3.01991574e-04, 9.99575958e-01]), array([1.30570982e-05, 8.11299880e-06, 9.99978830e-01]), array([0.39174818, 0.56076878, 0.04748304])]
Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [25]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

[array([0.01043134, 0.06006002, 0.01145332, 0.00460319, 0.00899074,
       0.00659741, 0.00588434, 0.00543873, 0.01913355, 0.00440472,
       0.00117287, 0.00498903, 0.00982399, 0.02170975, 0.00656725,
       0.00762632, 0.01871608, 0.00315683, 0.01490108, 0.00299942,
       0.00180181, 0.01017089, 0.00302964, 0.00757446, 0.00280221,
       0.00379319, 0.00894089, 0.00609675, 0.01479801, 0.00855455,
       0.01117655, 0.00342065, 0.00574597, 0.0052358 , 0.0066932 ,
       0.06960769, 0.00447415, 0.00371878, 0.00759784, 0.01160758,
       0.01144284, 0.00780362, 0.0217761 , 0.02548132, 0.00738601,
       0.01097032, 0.0083364 , 0.00317682, 0.0111048 , 0.01594194,
       0.01052272, 0.01329633, 0.00202779, 0.01358219, 0.00451156,
       0.00311991, 0.0050552 , 0.00628738, 0.00545589, 0.00816001,
       0.01323018, 0.00557249, 0.01257088, 0.00812513, 0.00680572,
       0.01944452, 0.01416098, 0.0194583 , 0.00839333, 0.00246675,
       0.0137056 , 0.00461931, 0.00608005, 0.01084456, 0.0109

That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.